# Create plaquette using the RPNG format

## 1. RPNG format

```
-z1- -z2- -z3- -z4-
rpng rpng rpng rpng
```

(r) data qubit reset basis or h or -  
(p) data basis for the controlled operation (x means CNOT controlled on the ancilla and targeting the data qubit, y means CY, z means CZ)  
(n) time step (positive integers, all distinct, typically in 1-5)  
(g) data qubit measure basis or h or -

Assumptions on the circuit:
- if not otherwise stated, a basis can be {x,y,z}
- the ancilla is always initialized in $\ket{+}$ and measured in the X basis
- the ancilla is always the control qubit for the CNOT and CZ gates
- time step of r same as ancilla reset (default 0)
- time step of g same as ancilla measurement (default 6)

## 2. Extended RPNG format

```
z0z5 -xz1- -xz2- -xz3- -xz4-
pnpn rppng rppng rppng rppng
```

(p) ancilla init basis  
(n) time step  
(p) ancilla measure basis  
(n) time step

(r) data qubit reset basis or h or -  
(pp) ancilla-data 2-qubit bases (xz means CNOT targeting the ancilla)
(n) time step (positive integers, all distinct)
(g) data qubit measure basis or h or -

Assumptions on the circuit:
- if not otherwise stated, a basis can be {x,y,z}
- at least one of the (pp) must be z, indicating the control qubit
- time step of r same as ancilla reset
- time step of g same as ancilla measurement
- the time step of every (pp) must be in [ancilla init time + 1, ancilla measure time -1]  

## 3. Test the functionality

In [1]:
from typing import Literal
from pathlib import Path
import stim
import matplotlib.pyplot as plt

from tqec import (
    Plaquette,
    PlaquetteQubits,
    QubitMap,
    ScheduledCircuit,
    SquarePlaquetteQubits,
    validate_rpng_string,
    create_plaquette_from_rpng_string,
)

ASSETS_FOLDER = Path("../../assets/").resolve()

In [2]:
rpng = '-x5h -z2z -x3x hz1-' # correct
qubits = SquarePlaquetteQubits()

format = validate_rpng_string(rpng)
if format == 1:
    print('-> RPNG is in simplified format\n')
elif format == 2:
    print('-> RPNG is in extended format\n')

try:
    plaq = create_plaquette_from_rpng_string(rpng_string = rpng, qubits = qubits)
except ValueError as err:
    print(f'{err}')

print('-> Circuit associated with the plaquette:\n')
print(plaq.circuit.get_circuit(include_qubit_coords=True))

-> RPNG is in simplified format

-> Circuit associated with the plaquette:

QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
H 3
RX 4
TICK
CZ 4 3
TICK
CZ 4 1
TICK
CX 4 2
TICK
TICK
CX 4 0
TICK
H 0
M 1
MX 2 4


In [3]:
# Simplified RPNG format.
rpng = '---- ---- ---- ---- ----' # wrong number of values
rpng = '---- ---- --- ----' # wrong length of values
rpng = '---- ---- ---- ----' # correct but uneventful
rpng = '-z1- -z2- -z3- -z4-' # correct
rpng = '-z1- -z2- ---- -z4-' # wrong number of 2Q gates
rpng = '-z1- -z4- -z3- -z4-' # wrong times for the 2Q gates
rpng = '-z1- -z6- -z3- -z4-' # wrong times for the 2Q gates
rpng = '-z5- -x2- -x3- -z1-' # correct
rpng = '-x5h -z2z -x3x hz1-' # correct

# Extended RPNG format.
rpng = 'z0z5 -xz1- -xz2- -xz6- -xz4-' # wrong times for the 2Q gates
rpng = 'z0z3 -xz1- ----- -xz2- -----' # correct
rpng = 'z3z0 -xx1- ----- -xz2- ----' # wrong meas time
rpng = 'z0z5 -xz1- -xz2- -xz3- -xz4-' # correct

try:
    plaq = create_plaquette_from_rpng_string(rpng_string = rpng)
    print(f'Circuit associated with the plaquette "{rpng}":\n')
    print(plaq.circuit.get_circuit(include_qubit_coords=True))
except ValueError as err:
    print(f'{err}')

Circuit associated with the plaquette "z0z5 -xz1- -xz2- -xz3- -xz4-":

QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
RX 4
TICK
CX 0 4
TICK
CX 1 4
TICK
CX 2 4
TICK
CX 3 4
TICK
MX 4


## 4. Example: associate a custom circuit to the plaquette

**TODO:** Here we just copied the code from the tqec implementation. Provide a different implementation or showcase a proper interface (still to be developed).

In [4]:
PLAQUETTE_PREP_TIME = 0
PLAQUETTE_MEAS_TIME = 6

def custom_create_plaquette_from_rpng_string(
        rpng_string: str,
        qubits: PlaquetteQubits,
        prep_time: int = PLAQUETTE_PREP_TIME,
        meas_time: int = PLAQUETTE_MEAS_TIME
    ) -> Plaquette:
    """Create a plaquette from the RPNG format
    
    Assumptions:
    - the ancilla qubit is the last among the PlaquetteQubits (thus has index 4)
    """
    format = validate_rpng_string(rpng_string, prep_time=prep_time, meas_time=meas_time)
    circuit_as_list = []
    if format == 1:
        print('simplified RPNG format')
        circuit_as_list = [''] * (meas_time - prep_time + 1)
        for q, v in enumerate(rpng_string.split(' ')):
            # 2Q gates.
            circuit_as_list[int(v[2])] += f'C{v[1].upper()} 4 {q}\n'
            # Data reset or Hadamard.
            if v[0] == 'h':
                circuit_as_list[0] += f'H {q}\n'
            elif v[0] != '-':
                circuit_as_list[0] += f'R{v[0].upper()} {q}\n'
            # Data measurement or Hadamard.
            if v[3] == 'h':
                circuit_as_list[-1] += f'H {q}\n'
            elif v[3] != '-':
                circuit_as_list[-1] += f'M{v[3].upper()} {q}\n'
        # Ancilla reset and measurement.
        circuit_as_list[0] += 'RX 4\n'
        circuit_as_list[-1] += 'MX 4\n'
    elif format == 2:
        print('extended RPNG format')
        # Update prep and meas times.
        values = rpng_string.split(' ')
        prep_time = int(values[0][1])
        meas_time = int(values[0][3])
        circuit_as_list = [''] * (meas_time - prep_time + 1)
        for q, v in enumerate(values[1:]):
            # 2Q gates.
            if v[1] == 'z':
                circuit_as_list[int(v[3])] += f'C{v[2].upper()} 4 {q}\n'
            elif v[2] == 'z':
                circuit_as_list[int(v[3])] += f'C{v[1].upper()} {q} 4\n'
            # Data reset or Hadamard.
            if v[0] == 'h':
                circuit_as_list[0] += f'H {q}\n'
            elif v[0] != '-':
                circuit_as_list[0] += f'R{v[0].upper()} {q}\n'
            # Data measurement or Hadamard.
            if v[4] == 'h':
                circuit_as_list[-1] += f'H {q}\n'
            elif v[4] != '-':
                circuit_as_list[-1] += f'M{v[4].upper()} {q}\n'
        # Ancilla reset and measurement.
        circuit_as_list[0] += 'RX 4\n'
        circuit_as_list[-1] += 'MX 4\n'
    elif format == 0:
        raise ValueError(f'invalide rpng string "{rpng_string}"')
    q_map = QubitMap.from_qubits(qubits)
    circuit_as_str = "TICK\n".join(circuit_as_list)
    circuit = stim.Circuit(circuit_as_str)
    scheduled_circuit = ScheduledCircuit.from_circuit(circuit, qubit_map = q_map)
    return Plaquette(name = 'test', qubits = qubits, circuit = scheduled_circuit)

rpng = '-x5h -z2z -x3x hz1-' # correct
qubits = SquarePlaquetteQubits()

try:
    plaq = custom_create_plaquette_from_rpng_string(rpng_string = rpng, qubits = qubits)
    print(plaq.circuit.get_circuit(include_qubit_coords=True))
except ValueError as err:
    print(f'{err}')

simplified RPNG format
QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
H 3
RX 4
TICK
CZ 4 3
TICK
CZ 4 1
TICK
CX 4 2
TICK
TICK
CX 4 0
TICK
H 0
M 1
MX 2 4


----

## 5. Rpng as dataclass

In [52]:
from dataclasses import dataclass, field
from enum import Enum
from typing import Optional, Literal, Mapping

from tqec.templates.base import Template


class BasisEnum(Enum):
    X = 'x'
    Y = 'y'
    Z = 'z'

class ExtendedBasisEnum(Enum):
    X = BasisEnum.X.value
    Y = BasisEnum.Y.value
    Z = BasisEnum.Z.value
    H = 'h'


@dataclass
class RPNG:
    r: Optional[ExtendedBasisEnum]
    p: Optional[BasisEnum]
    n: Optional[int]
    g: Optional[ExtendedBasisEnum]
    a: BasisEnum = BasisEnum.Z

    @classmethod
    def from_string(cls, rpng_string: str) -> 'RPNG':
        """Initialize the RPNG object from a 4-character string
        
        Note that any character different from a BasisEnum / ExtendedBasisEnum
        value would result in the corresponding field being None.
        """
        if len(rpng_string) != 4:
            raise ValueError('The rpng string must be exactly 4-character long.')
        r_str, p_str, n_str, g_str = tuple(rpng_string)
        r = ExtendedBasisEnum(r_str) if r_str in ExtendedBasisEnum._value2member_map_ else None
        p = BasisEnum(p_str) if p_str in BasisEnum._value2member_map_ else None
        n = int(n_str) if n_str.isdigit() else None
        g = ExtendedBasisEnum(g_str) if g_str in ExtendedBasisEnum._value2member_map_ else None
        return cls(r, p, n, g)


@dataclass
class RGN:
    """Organize the prep and meas bases for the ancilla, together with the meas time
    
    The init time is assumed to be 0.
    """
    r: BasisEnum = BasisEnum.X
    g: BasisEnum = BasisEnum.X
    n: int = 6

    @classmethod
    def from_string(cls, rgn_string: str) -> 'RGN':
        """Initialize the RGN object from a 3-character string"""
        if len(rgn_string) != 3:
            raise ValueError('The rgn string must be exactly 3-character long.')
        r_str, g_str, n_str = tuple(rgn_string)

        try:
            r = BasisEnum(r_str)
            g = BasisEnum(g_str)
            n = BasisEnum(n_str)
            return cls(r, g, n)
        except ValueError:
            raise ValueError('Invalid rgn string.')


@dataclass
class RPNGDescription:
    """Organize the description of a plaquette in RPNG format
    
    The corners of the square plaquette are listed following the order:
    tl, tr, bl, br
    """
    corners: tuple[RPNG, RPNG, RPNG, RPNG]
    ancilla: RGN = RGN()

    def __post_init__(self):
        """Validation of the initialization arguments
        
        Constraints:
        - the n values for the corners must be unique
        - the n values for the corners must be in the interval ]0, ancilla.n[
        """
        times = []
        for rpng in self.corners:
            times.append(rpng.n)
        if len(times) != len(set(times)):
            raise ValueError('The n values for the corners must be unique')
        elif len(times) not in [0,2,4]:
            raise ValueError('Each plaquette must have 0, 2, or 4 2Q gates')

    def get_r_op(self, data_idx: int) -> Optional[str]:
        """Get the reset operation or Hadamard for the specific data qubit"""
        op = self.corners[data_idx].r
        if op == None:
            return None
        elif op.value in BasisEnum._value2member_map_:
            return f'R{op.value.upper()}'
        else:
            return f'{op.value.upper()}'

    def get_n(self, data_idx: int):
        """Get the time of the 2Q gate involving the specific data qubit"""
        return self.corners[data_idx].n

    def get_g_op(self, data_idx: int) -> Optional[str]:
        """Get the reset operation or Hadamard for the specific data qubit"""
        op = self.corners[data_idx].g
        if op == None:
            return None
        elif op.value in BasisEnum._value2member_map_:
            return f'M{op.value.upper()}'
        else:
            return f'{op.value.upper()}'
        



BasisEnum.X.value == 'X'
a = RPNG(None, BasisEnum.X, 1, ExtendedBasisEnum.H)
print(a)
b = RPNG.from_string('xx4-')
print(b)

desc = RPNGDescription((a,a,a,b))
print(desc)
print(desc.get_r_op(0), desc.get_r_op(1), desc.get_r_op(2), desc.get_r_op(3))
print(desc.get_g_op(0), desc.get_g_op(1), desc.get_g_op(2), desc.get_g_op(3))

RPNG(r=None, p=<BasisEnum.X: 'x'>, n=1, g=<ExtendedBasisEnum.H: 'h'>, a=<BasisEnum.Z: 'z'>)
RPNG(r=<ExtendedBasisEnum.X: 'x'>, p=<BasisEnum.X: 'x'>, n=4, g=None, a=<BasisEnum.Z: 'z'>)


ValueError: The n values for the corners must be unique

In [55]:
full_rpng = '-x5h -z2z -x3x hz1-'
corners = tuple([RPNG.from_string(k) for k in full_rpng.split(' ')])
desc = RPNGDescription(corners)
print(desc)

RPNGDescription(corners=(RPNG(r=None, p=<BasisEnum.X: 'x'>, n=5, g=<ExtendedBasisEnum.H: 'h'>, a=<BasisEnum.Z: 'z'>), RPNG(r=None, p=<BasisEnum.Z: 'z'>, n=2, g=<ExtendedBasisEnum.Z: 'z'>, a=<BasisEnum.Z: 'z'>), RPNG(r=None, p=<BasisEnum.X: 'x'>, n=3, g=<ExtendedBasisEnum.X: 'x'>, a=<BasisEnum.Z: 'z'>), RPNG(r=<ExtendedBasisEnum.H: 'h'>, p=<BasisEnum.Z: 'z'>, n=1, g=None, a=<BasisEnum.Z: 'z'>)), ancilla=RGN(r=<BasisEnum.X: 'x'>, g=<BasisEnum.X: 'x'>, n=6))
